In [ ]:
WEB_PUB_SUB_CONN_STRING = "Endpoint=https://ssattiraju-multi-agent.webpubsub.azure.com;AccessKey=3pKQvCImpSBwDYYUWxjkvmp9V3v6CGLnjbjrWBGf4LlZPv8mDpFzJQQJ99AKACfhMk5XJ3w3AAAAAWPS3qWX;Version=1.0;"
WEB_PUB_SUB_HUB_NAME = "Hub"

In [ ]:
import asyncio
import os
import socketio
import nest_asyncio
from azure.messaging.webpubsubservice import WebPubSubServiceClient

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Read from environment variables
WEB_PUB_SUB_CONN_STRING = "Endpoint=https://ssattiraju-multi-agent.webpubsub.azure.com;AccessKey=3pKQvCImpSBwDYYUWxjkvmp9V3v6CGLnjbjrWBGf4LlZPv8mDpFzJQQJ99AKACfhMk5XJ3w3AAAAAWPS3qWX;Version=1.0;"
WEB_PUB_SUB_HUB_NAME = "Hub"
# Initialize Socket.IO client
sio = socketio.AsyncClient()


@sio.event
async def connect():
    print("connected to server")
    # Simulate sending a message after connecting
    await sio.emit("message", "Hello, Server!")


@sio.event
async def disconnect():
    print("disconnected from server")


@sio.event
async def message(data):
    print("Message received: " + data)


async def main():
    service = WebPubSubServiceClient.from_connection_string(
        connection_string=WEB_PUB_SUB_CONN_STRING, hub=WEB_PUB_SUB_HUB_NAME
    )
    token = service.get_client_access_token()
    new_url = token["url"].replace("/client/hubs/", "/clients/socketio/hubs/")
    print(new_url)
    await sio.connect(new_url, transports=["websocket"])
    await sio.wait()


try:
    asyncio.get_event_loop().run_until_complete(main())
except KeyboardInterrupt:
    pass

In [ ]:
from azure.messaging.webpubsubservice import WebPubSubServiceClient
import datetime
import logging
import sys

logger = logging.getLogger("azure")
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(stream=sys.stdout)
logger.addHandler(handler)

service = WebPubSubServiceClient.from_connection_string(
    "Endpoint=https://ssattiraju-multi-agent.webpubsub.azure.com;AccessKey=3pKQvCImpSBwDYYUWxjkvmp9V3v6CGLnjbjrWBGf4LlZPv8mDpFzJQQJ99AKACfhMk5XJ3w3AAAAAWPS3qWX;Version=1.0;",
    hub="Hub",
)
service.send_to_all(
    message={
        "from": "user1",
        "data": "Hello world 1234 sent at " + str(datetime.datetime.now()),
    },
    logging_enable=True,
)
service.send_to_connection("user1", message="hello from sunil")

In [ ]:
from azure.messaging.webpubsubclient import WebPubSubClient, WebPubSubClientCredential
from azure.messaging.webpubsubservice import WebPubSubServiceClient
from azure.messaging.webpubsubclient.models import (
    OnConnectedArgs,
    OnGroupDataMessageArgs,
    OnDisconnectedArgs,
    CallbackType,
    WebPubSubDataType,
)

client = WebPubSubClient(
    "Endpoint=https://ssattiraju-multi-agent.webpubsub.azure.com;AccessKey=3pKQvCImpSBwDYYUWxjkvmp9V3v6CGLnjbjrWBGf4LlZPv8mDpFzJQQJ99AKACfhMk5XJ3w3AAAAAWPS3qWX;Version=1.0;"
)

In [ ]:
client.subscribe(
    CallbackType.SERVER_MESSAGE, lambda e: print(f"Received message {e.data}")
)

In [ ]:
import os
from azure.messaging.webpubsubclient import WebPubSubClient, WebPubSubClientCredential
from azure.messaging.webpubsubservice import WebPubSubServiceClient
from azure.messaging.webpubsubclient.models import (
    OnConnectedArgs,
    OnGroupDataMessageArgs,
    OnDisconnectedArgs,
    CallbackType,
    WebPubSubDataType,
)


def on_group_message(msg: OnGroupDataMessageArgs):
    print("========== group message =========")
    if isinstance(msg.data, memoryview):
        print(f"Received message from {msg.group}: {bytes(msg.data).decode()}")
    else:
        print(f"Received message from {msg.group}: {msg.data}")


def main():
    service_client = WebPubSubServiceClient.from_connection_string(  # type: ignore
        connection_string="Endpoint=https://ssattiraju-multi-agent.webpubsub.azure.com;AccessKey=3pKQvCImpSBwDYYUWxjkvmp9V3v6CGLnjbjrWBGf4LlZPv8mDpFzJQQJ99AKACfhMk5XJ3w3AAAAAWPS3qWX;Version=1.0;",
        hub="Hub",
    )
    client = WebPubSubClient(
        credential=WebPubSubClientCredential(
            client_access_url_provider=lambda: service_client.get_client_access_token(
                roles=["webpubsub.joinLeaveGroup", "webpubsub.sendToGroup"]
            )["url"]
        ),
    )
    print(client.is_connected())

    with client:
        client.subscribe(CallbackType.CONNECTED, on_connected)
        client.subscribe(CallbackType.DISCONNECTED, on_disconnected)
        client.subscribe(CallbackType.GROUP_MESSAGE, on_group_message)
        group_name = "hello_world"
        client.join_group(group_name)
        client.send_to_group(
            group_name, "hello text", WebPubSubDataType.TEXT, no_echo=False, ack=False
        )
        client.send_to_group(group_name, {"hello": "json"}, WebPubSubDataType.JSON)
        client.send_to_group(group_name, "hello text", WebPubSubDataType.TEXT)
        content = memoryview("hello binary".encode())
        client.send_to_group(group_name, content, WebPubSubDataType.BINARY)

    # If you can't run client in context, please open/close client manually like:
    # client.open()
    # ...
    # client.close()


if __name__ == "__main__":
    main()

In [ ]:
export WebPubSubConnectionString="Endpoint=https://ssattiraju-multi-agent.webpubsub.azure.com;AccessKey=3pKQvCImpSBwDYYUWxjkvmp9V3v6CGLnjbjrWBGf4LlZPv8mDpFzJQQJ99AKACfhMk5XJ3w3AAAAAWPS3qWX;Version=1.0;"

In [ ]:
from t
async def create_hotel_booking(
    city: Annotated[str, "The city where the hotel booking will take place."],
    check_in_date: Annotated[
        str, "The check-in date of the hotel booking in the format 'YYYY-MM-DD'."
    ],
    check_out_date: Annotated[
        str, "The check-out date of the hotel booking in the format 'YYYY-MM-DD'."
    ],
) -> Dict[str, str | int]:
    # Simulate available hotel options
    logger.info(
        f"Function call: Creating hotel booking for {city} from {check_in_date} to {check_out_date}"
    )
    hotel_options = [
        {"hotel_name": "Hilton", "room_type": "Deluxe", "price_per_night": 200},
        {"hotel_name": "Marriott", "room_type": "Standard", "price_per_night": 150},
        {"hotel_name": "Hyatt", "room_type": "Suite", "price_per_night": 300},
        {"hotel_name": "Sheraton", "room_type": "Executive", "price_per_night": 250},
        {"hotel_name": "Holiday Inn", "room_type": "Standard", "price_per_night": 100},
        {"hotel_name": "Ritz-Carlton", "room_type": "Luxury", "price_per_night": 400},
        {
            "hotel_name": "Four Seasons",
            "room_type": "Presidential Suite",
            "price_per_night": 500,
        },
        {
            "hotel_name": "InterContinental",
            "room_type": "Club Room",
            "price_per_night": 350,
        },
        {"hotel_name": "Westin", "room_type": "Deluxe", "price_per_night": 220},
        {"hotel_name": "Radisson", "room_type": "Standard", "price_per_night": 180},
    ]

    # Randomly select a hotel option
    selected_hotel = random.choice(hotel_options)

    # Calculate the number of nights
    check_in = datetime.datetime.strptime(check_in_date, "%Y-%m-%d")
    check_out = datetime.datetime.strptime(check_out_date, "%Y-%m-%d")
    num_nights = (check_out - check_in).days

    # Calculate total price for the stay
    total_price = num_nights * selected_hotel["price_per_night"]

    # Create a booking reference number
    booking_reference = f"HT-{random.randint(1000, 9999)}-{city[:3].upper()}"

    # Simulate hotel booking details
    hotel_booking_details = {
        "city": city,
        "check_in_date": check_in_date,
        "check_out_date": check_out_date,
        "hotel_name": selected_hotel["hotel_name"],
        "room_type": selected_hotel["room_type"],
        "total_price": total_price,
        "booking_reference": booking_reference,
    }
    logger.info(f"Hotel booking details: {hotel_booking_details}")

    return hotel_booking_details


def get_hotel_booking_tool() -> List[Tool]:
    return [
        FunctionTool(
            create_hotel_booking,
            description="This function performs the process of booking a hotel. It takes the city, check-in date, and check-out date as input and returns the hotel booking details, including the city, check-in and check-out dates, hotel name, room type, total price, and booking reference. This function is useful when the user wants to book a hotel",
        )
    ]

ImportError: attempted relative import beyond top-level package